In [1]:
from src import data_loading
from src import preprocessing
from src import feature_engenieering
from src import modeling
from src import evaluation

#files path for the raw dataset:
data_files = ['./data/people.csv','./data/descriptions.csv','./data/salary.csv',]

#merge datasets in a cohesive Dataframe
full_dataset = data_loading.load_data(data_files)

#preprocessing of the dataframe adds missing values with LLM inference over descriptions of each row, drops the incomplete rows and cleans up the data.
cleansed_dataset = await preprocessing.preprocess(full_dataset)

#split the dataset into an 80 / 20 ratio for training and testing.
X_train, X_test, y_train, y_test = feature_engenieering.split_data(cleansed_dataset)


#normalize and scale the datasets using MinMaxScaler and target encoder

normalized_X_train, te, scaler = feature_engenieering.normalize_train_data(X_train, y_train)

normalized_X_test = feature_engenieering.normalize_test_data(X_test, te, scaler)



2025-01-12 18:43:14.963656: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:matplotlib:matplotlib data path: /home/haervwe/Predictive_Models/salary_predictive_model/.venv/lib/python3.12/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/haervwe/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/haervwe/.cache


Full Merged Dataset:
   id   Age  Gender Education Level          Job Title  Years of Experience  \
0   0  32.0    Male      Bachelor's  Software Engineer                  5.0   
1   1  28.0  Female        Master's       Data Analyst                  3.0   
2   2  45.0    Male             PhD     Senior Manager                 15.0   
3   3  36.0  Female      Bachelor's    Sales Associate                  7.0   
4   4  52.0    Male        Master's           Director                 20.0   

                                         Description    Salary  
0  I am a 32-year-old male working as a Software ...   90000.0  
1  I am a 28-year-old data analyst with a Master'...   65000.0  
2  I am a 45-year-old Senior Manager with a PhD a...  150000.0  
3  I am a 36-year-old female Sales Associate with...   60000.0  
4  I am a 52-year-old male with over two decades ...  200000.0  
      id   Age  Gender Education Level                      Job Title  \
370  370  35.0  Female      Bachelor's  

DEBUG:src.llm_dataset_filler:Index 260: Received NDJSON response for Job Title.
DEBUG:src.llm_dataset_filler:Index 260: Received NDJSON response for Age.
DEBUG:src.llm_dataset_filler:Index 172: Received NDJSON response for Years of Experience.
DEBUG:src.llm_dataset_filler:Index 260: Received NDJSON response for Gender.
DEBUG:src.llm_dataset_filler:Index 260: Inferred Job Title: Notfound
DEBUG:src.llm_dataset_filler:Index 260: Inferred Age: Notfound
DEBUG:src.llm_dataset_filler:Index 172: Inferred Years of Experience: Notfound
DEBUG:src.llm_dataset_filler:Index 260: Inferred Gender: Notfound
DEBUG:src.llm_dataset_filler:Index 60: Received NDJSON response for Job Title.
DEBUG:src.llm_dataset_filler:Index 260: Received NDJSON response for Years of Experience.
DEBUG:src.llm_dataset_filler:Index 260: Received NDJSON response for Education Level.
DEBUG:src.llm_dataset_filler:Index 221: Received NDJSON response for Age.
DEBUG:src.llm_dataset_filler:Index 60: Inferred Job Title: DirectorofOper


Missing Values in Full Dataset:
id                     0
Age                    3
Gender                 0
Education Level        0
Job Title              0
Years of Experience    2
Description            3
Salary                 2
dtype: int64

Rows with Missing Values:
      id   Age    Gender Education Level                 Job Title  \
111  111  37.0      Male      Bachelor's  Software Project Manager   
125  125  26.0      Male      Bachelor's         Junior Accountant   
172  172   NaN  Notfound        Notfound                  Notfound   
177  177  31.0      Male      Bachelor's         Junior Accountant   
260  260   NaN  Notfound        Notfound                  Notfound   
315  315   NaN      Male      Bachelor's  Senior Software Engineer   

     Years of Experience                                        Description  \
111                  9.0                                                NaN   
125                  2.0                                                NaN   

In [2]:
#train the model using a random forest regressor algorithm and print out the predictions for the normalized test data.

model = modeling.train_model(normalized_X_train, y_train)

#use the test dataset to predict salaries based on the trained model.




print(evaluation.evaluate_model(normalized_X_test, y_test, normalized_X_train,y_train, model))


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=1.0, min_samples_leaf=1, min_samples_split=10, n

ValueError: Found input variables with inconsistent numbers of samples: [297, 75]